# 📊 Intrade.bar Trading Data Analysis
This notebook analyzes profitability across symbols, time of day, day of week, and trade direction based on Intrade.bar export data.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.ticker as mtick

# Matplotlib display inline
%matplotlib inline

In [ ]:
df = pd.read_csv("csv/deriv_export.csv", sep=',')

# Clean numeric columns
#for col in ["Amount", "Result"]:
#    df[col] = df[col].str.replace(" USD", "").astype(float)


In [ ]:
from datetime import datetime

def parse_custom_datetime(x):
    return datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S")

df["entry_time"] = df["purchase_time"].apply(parse_custom_datetime)
df["exit_time"] = df["sell_time"].apply(parse_custom_datetime)

def convert_side(x):
    return "Up" if x == "CALL" else "Down"

df["side"] = df["contract_type"].apply(convert_side)


In [ ]:
df["symbol"] = df["underlying_symbol"]
df["pnl"] = df["sell_price"] - df["buy_price"]
df["hour"] = df["entry_time"].dt.hour
df["day_of_week"] = df["entry_time"].dt.day_name()


In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="symbol", y="pnl", data=df, estimator=sum)
plt.title("Total Profit/Loss by Symbol")
plt.ylabel("Total PnL")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="hour", y="pnl", data=df, estimator=sum, palette="viridis", hue="hour", legend=False)
plt.title("Profit/Loss by Hour of Day")
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

In [ ]:
order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
plt.figure(figsize=(10, 5))
sns.barplot(x="day_of_week", y="pnl", data=df, estimator=sum, order=order)
plt.title("Profit/Loss by Day of Week")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 5))
sns.barplot(x="side", y="pnl", data=df, estimator=sum, palette="Set2", hue="side", legend=False)
plt.title("PnL by Side (Up/Down)")
plt.tight_layout()
plt.show()

In [ ]:
pivot = df.pivot_table(index="day_of_week", columns="hour", values="pnl", aggfunc="sum")
pivot = pivot.reindex(order)
plt.figure(figsize=(12, 6))
sns.heatmap(pivot, cmap="coolwarm", center=0, annot=True, fmt=".0f")
plt.title("PnL Heatmap by Day and Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Day of Week")
plt.tight_layout()
plt.show()

In [ ]:
fig = px.scatter_3d(
    df,
    x="symbol",
    y="hour",
    z="pnl",
    color="symbol",
    title="3D: Symbol vs Hour vs PnL"
)
fig.update_layout(width=1400, height=1000)
fig.show()